## 有监督学习
我们的目标是依据某个带标注信息的输入数据集（即其中的每个样本都标注了真实的或期望的输出）去训练一个推断模型。该模型能覆盖一个数据集，并可对不存在于初始训练集中的新样本的输出进行预测。

**推断模型**：即运用到数据集上的一系列数学运算。具体的运算步骤是通过代码设置的，并由用于求解某个给定问题的模型确定。模型确定后，构建模型的运算也就固定了。在各运算内部，有一些与其定义相关的数值，如“乘以3”、“加2”。这些值都是模型的参数，且在训练过程中需要不断更新，以让模型可以学习，并对输出进行调整。

**数据流图的高层、通用训练闭环**：
- *对模型参数进行初始化*：
    - 通常采用对参数随机赋值的方法，但对于比较简单的模型，也可将各个参数的初值设为0。
- *读取训练数据*（包括每个数据样本及其期望输出）：
    - 通常人们会在这些数据被送入模型之前，随机打乱样本的次序。
- *在训练数据上执行推断模型*：
    - 这样，在当前模型参数配置下，每个训练样本都会得到一个输出值。
- *计算损失*：
    - 损失是一个能够刻画模型在最后一步得到的输出与来自训练集的期望输出之间的差距的概括性指标。
- *调整模型参数*：
    - 此步对应于实际的学习过程。给定损失函数，学习的目的在于通过大量训练步骤改善个参数的值来使得损失最小化。
    
上述闭环会依据所需的学习率、所给定的模型及其输入数据，通过大量循环不断重复上述过程。

当训练结束后，便进入**评估阶段**：
    我们需要对一个同样含有期望输出信息的不同测试集依据模型进行推断，并评估模型在该数据集上的损失。     
    最常见的方法是：将原始数据集一分为二，$70%$ 作为训练，其余$30%$作为评估。

# 模型训练和评估的通用代码框架：
```python
import tensorflow as tf

# 初始化变量和模型参数，定义训练闭环中的运算

def inference(X):
    '''计算推断模型在数据集X上的输出，并将结果返回'''
    pass

def loss(X, Y):
    '''
    依据训练数据X及其期望输出Y计算损失
    '''
    pass

def inputs():
    '''
    读取或生成训练数据X及其期望生成Y
    '''
    pass

def train(total_loss):
    '''依据计算的总损失训练或调整模型参数'''
    pass

def evaluate(sess, X, Y):
    '''对训练得到的模型进行评估'''
    pass

# 在一个会话对象中启动数据流图，搭建流程
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord= coord)
    
    # 实际的训练迭代次数
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # 出于调试和学习的目的，查看损失在训练过程的递减情况
        if step % 10 == 0:
            print('loss:', sess.run([total_loss]))
            
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)
```

## 保存训练检查点
训练模型意味着通过许多个训练周期更新其参数（即Variable对象/变量）。由于变量都保存在内存中，所以若计算机经历长时间的训练后突然断电，所有的文件都将丢失。为此，我们需要借助 `tf.train.Saver` 类将数据流图中的变量保存在专门的二进制文件中。我们应当周期性地保存所有变量，创建检查点（checkpoint）文件，并在必要时从最近的检查点恢复训练。

为使用Saver类，需要对之前的训练闭环代码框架略作修改：
```python
import tensorflow as tf

# 初始化变量和模型参数，定义训练闭环中的运算

def inference(X):
    '''计算推断模型在数据集X上的输出，并将结果返回'''
    pass

def loss(X, Y):
    '''
    依据训练数据X及其期望输出Y计算损失
    '''
    pass

def inputs():
    '''
    读取或生成训练数据X及其期望生成Y
    '''
    pass

def train(total_loss):
    '''依据计算的总损失训练或调整模型参数'''
    pass

def evaluate(sess, X, Y):
    '''对训练得到的模型进行评估'''
    pass

# 创建一个 Saver 对象 
saver = tf.train.Saver()

# 在一个会话对象中启动数据流图，搭建流程
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord= coord)

    # 实际的训练迭代次数
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        
        # 出于调试和学习的目的，查看损失在训练过程的递减情况
        if step % 10 == 0:
            print('loss:', sess.run([total_loss]))
            saver.save(sess, 'xinet-model', global_step = step)

    evaluate(sess, X, Y)

    coord.request_stop()
    coord.join(threads)
    
    saver.save(sess, 'xinet-model', global_step = training_steps)
```

每次调用 `tf.train.Saver.save` 方法都将创建一个遵循命名模板$xinet-model-\{step\}$ 的检查点文件（如$xinet-model-100$）。该文件会保存每个变量的当前值。默认情况下，Saver 对象只会保留最近的5个文件，更早的文件都将被自动删除。

如果希望从某个检查点恢复训练，则应使用 `tf.train.get_checkpoint_state`方法，以验证之前是否有检查点文件被保存下来，而 `tf.train.Saver.restore` 方法将负责恢复变量的值。
```python
with tf.Session() as sess:
    # 模型设置······
    initial_step = 0
    
    # 验证之前是否已经保存了检查点文件
    ckpt = tf.train.get_checkpoint_state(os.path.dirname(__file__))
    if ckpt and ckpt.model_checkpoint_path:
        # 从检查点恢复模型参数
        saver.restore(sess, ckpt.model_checkpoint_path)
        initial_step = int(ckpt.model_checkpoint_path.rsplit('-', 1)[1])
        
        # 实际的训练闭环
        for step in range(initial_step, training_steps): 
            ···
```

In [13]:
class Xinet(object):
    '''
     使用TensorFlow使用计算图（有向无环图），支持并行运算：
        准备样本数据集、变量、占位符和机器学习模型，然后进行模型训练，更新变量状态来提高预测结果。
        创建损失函数，通过调整计算图中的变量来最小化损失函数。
        维护模型的计算状态，每步迭代自动计算梯度。
    '''
    
    def __init__(self, *data):
        '''
        导入/生成样本数据集
        '''
        import tensorflow as tf
        self.tf = tf
        self.data = data
        
    def transform_normalizer(self):
        '''
        转换和归一化数据：
            一般地，输入样本数据集并不符合 TensorFlow 期望的`shape`，所以需要转换数据格式以满足TensorFlow。
            当数据集的维度和类型不符合所使用的机器学习算法的要求时，需要在使用前进行数据转换。
            大部分机器学习算法期待的输入样本是归一化的数据。
            
            例子：`data = tf.nn.batch_norm_with_global_normalization(...)`
        '''
        pass
    
      def divide_data(self):
        '''
        划分数据集为训练集、验证集和测试集。
        一般要求机器学习算法的训练集和测试集是不同的数据集。
        验证集是用来进行超参数调优的。
        '''
        pass
    
    def set_parameter(self):
        '''
        设置机器学习的参数和超参数。
        超参数：如迭代次数、学习率或其他固定参数。约定俗成的习惯是一次性初始化所有的机器学习参数，如：
            learning_rate = 0.01
            batch_size = 100
            iterations = 1000
            ...
        '''
        pass
    
    def init_variablesOp(self):
        '''
        初始化变量和占位符:
                指定数据大小和数据类型初始化变量和占位符。
            一般使用`tf.float32`,使用的数据类型的字节数越多结果越精确，同时运行速度越慢。
        '''
        pass
    
    def model(self):
        '''
        在获取样本数据集、初始化变量和占位符后，需要定义机器学习模型。
        TensorFlow通过选择`Op`、变量和占位符来构建计算图。
        如 `y_pred = tf.add(tf.matmul(x_input, weight), b)`
        '''
        pass
    
    def loss(self):
        '''
        定义完模型后，需要声明损失函数来评估输出结果。
        '''
        pass
    
    def train_model(self):
        '''
        初始化模型并训练模型:
            创建计算图实例，通过占位符赋值，维护变量的状态信息。
            
        示例：
        ```
        with tf.Session(graph = g) as sess:
            ...
            sess.run(...)
            ...  
        ```
        '''
        pass
    
    def evaluate_model(self):
        '''
        一旦构建计算图，并训练机器学习模型后，需要寻找某种标准来评估模型对新样本数据集的效果：
            通过训练集和测试集的评估来确定模型是过拟合还是欠拟合。
        '''
        pass
    
    def prior_hyperParameter(self):
        '''
        大部分情况下，机器学习者需要基于模型效果来回调整一些超参数：
                通过调整不同的超参数来重复训练模型，并用验证集来评估机器学习模型。
        '''
        pass
        
    def publish_or_prediction(self):
        '''
        发布和预测结果：
            所有机器学习模型一旦训练好，最后都用来预测新的、未知的数据。
        '''
        pass

In [14]:
model = Xinet()

# 线性回归
$Y = XW + b$
   - $X = (x_1, x_2, \cdots, x_k)$：x_1, x_2, \cdots, x_k是一组独立的预测变量；
   - $W = (w_1, w_2, \cdots, w_k)$：模型从训练集中学习到的参数, 常被称为“权重（$weights$）”；
   - $b$：模型从训练集中学习到的参数，常被称为“偏置（$bias$）”。

## 1. 初始化变量（模型参数）

In [21]:
W = tf.Variable(tf.zeros([2, 1]), name= 'weights')
b = tf.Variable(0., name= 'bias')

def inference(X):
    return tf.matmul(X, W) + b

## 2. 定义损失函数

对于简单的模型一般采用总平方误差，即模型对每个训练样本的预测值与期望输出之差的平方的总和。对于2D数据集，总平方误差对应于每个数据点在垂直方向上到所预测的回归直线的距离的平方总和。这种损失函数也称为$L2$范数或$L2$损失函数。即：
$$loss = \sum_i\left(y_i - y_{predicted_i} \right)^2$$

- $i$：数据样本的索引

In [46]:
def loss(X, Y):
    Y_predicted = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y, Y_predicted))  # 平方差之和

## 3. 训练模型

In [52]:
def inputs():
    # Data from http://people.sc.fsu.edu/~jburkardt/datasets/regression/x09.txt
    weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], 
                  [63, 28], [72, 36], [79, 57], [75, 44], [27, 24], [89, 31], 
                  [65, 52], [57, 23], [59, 60], [69, 48], [60, 34], [79, 51], 
                  [75, 50], [82, 34], [59, 46], [67, 23], [85, 37], [55, 40], [63, 30]]
    blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 
                         365, 209, 290, 346, 254, 395, 434, 220, 374, 
                         308, 220, 311, 181, 274, 303, 244]

    return tf.to_float(weight_age), tf.to_float(blood_fat_content)

def train(total_loss):
    '''梯度下降方法'''
    learning_rate = 0.0000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

def evaluate(sess, X, Y):
    print("inference([[80., 25.]]):", sess.run(inference([[80., 25.]])))
    print("inference([[65., 25.]]):", sess.run(inference([[65., 25.]])))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    X, Y = inputs()
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        if step % 100 == 0:
            print("loss: ", sess.run([total_loss]))

    evaluate(sess, X, Y)

    coord.request_stop()
    coord.join(threads)

loss:  [7608772.0]
loss:  [5341925.0]
loss:  [5339993.0]
loss:  [5338746.5]
loss:  [5337538.5]
loss:  [5336334.0]
loss:  [5335130.0]
loss:  [5333926.0]
loss:  [5332724.5]
loss:  [5331523.0]
inference([[80., 25.]]): [[ 320.64968872]]
inference([[65., 25.]]): [[ 267.78182983]]
